# December 22, 2021

https://adventofcode.com/2021/day/22

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import datetime
import re

In [2]:
def pnow():
    print( datetime.datetime.now().isoformat() )

In [3]:
with open("../data/2021/22.txt", "r") as f:
    data = f.read()

In [4]:
mini = f'''on x=10..12,y=10..12,z=10..12
on x=11..13,y=11..13,z=11..13
off x=9..11,y=9..11,z=9..11
on x=10..10,y=10..10,z=10..10'''

In [5]:
test = f'''on x=-20..26,y=-36..17,z=-47..7
on x=-20..33,y=-21..23,z=-26..28
on x=-22..28,y=-29..23,z=-38..16
on x=-46..7,y=-6..46,z=-50..-1
on x=-49..1,y=-3..46,z=-24..28
on x=2..47,y=-22..22,z=-23..27
on x=-27..23,y=-28..26,z=-21..29
on x=-39..5,y=-6..47,z=-3..44
on x=-30..21,y=-8..43,z=-13..34
on x=-22..26,y=-27..20,z=-29..19
off x=-48..-32,y=26..41,z=-47..-37
on x=-12..35,y=6..50,z=-50..-2
off x=-48..-32,y=-32..-16,z=-15..-5
on x=-18..26,y=-33..15,z=-7..46
off x=-40..-22,y=-38..-28,z=23..41
on x=-16..35,y=-41..10,z=-47..6
off x=-32..-23,y=11..30,z=-14..3
on x=-49..-5,y=-3..45,z=-29..18
off x=18..30,y=-20..-8,z=-3..13
on x=-41..9,y=-7..43,z=-33..15
on x=-54112..-39298,y=-85059..-49293,z=-27449..7877
on x=967..23432,y=45373..81175,z=27513..53682'''

# Part 1

In [6]:
class Cuboid():
    # ranges in ?lim are inclusive

    def __init__( self, xlim, ylim, zlim, switch ):
        self.xlim = xlim
        self.ylim = ylim
        self.zlim = zlim
        self.switch = switch

    def size( self ):
        return (self.xlim[1] - self.xlim[0] + 1) * (self.ylim[1] - self.ylim[0] + 1) * (self.zlim[1] - self.zlim[0] + 1)
    
    def intersect( self, other ):
        xlim = [ max(self.xlim[0], other.xlim[0]), min(self.xlim[1], other.xlim[1]) ]
        ylim = [ max(self.ylim[0], other.ylim[0]), min(self.ylim[1], other.ylim[1]) ]
        zlim = [ max(self.zlim[0], other.zlim[0]), min(self.zlim[1], other.zlim[1]) ]

        # edge case: empty intersection
        if xlim[0] > xlim[1] or ylim[0] > ylim[1] or zlim[0] > zlim[1]:
            return None
        
        return Cuboid( xlim, ylim, zlim, self.switch )
    
    def __str__(self):
        return self.switch + " x=" + str(self.xlim[0]) + ".." + str(self.xlim[1]) + ",y=" + str(self.ylim[0]) + ".." + str(self.ylim[1]) + ",z" + str(self.zlim[0]) + ".." + str(self.zlim[1])
    
    def __repr__(self):
        return str(self)
    
    def remove_intersection( self, isct ):

        # edge case: empty intersection
        if isct is None:
            return self # I think shallow copy suffices
        
        # Return up to 6 disjoint cuboids for the volumes outside the intersection
        volumes = []

        if isct.xlim == self.xlim and isct.ylim == self.ylim and isct.zlim == self.zlim:
            return volumes
        
        # 1. Top
        if isct.zlim[1] < self.zlim[1]:
            volumes.append( Cuboid( self.xlim, self.ylim, [isct.zlim[1]+1, self.zlim[1]], self.switch ) )

        # 2. Bottom
        if isct.zlim[0] > self.zlim[0]:
            volumes.append( Cuboid( self.xlim, self.ylim, [self.zlim[0], isct.zlim[0]-1], self.switch ) )

        # 3. Front, but not above/below
        if isct.ylim[1] < self.ylim[1]:
            volumes.append( Cuboid( self.xlim, [isct.ylim[1]+1, self.ylim[1]], isct.zlim, self.switch ) )

        # 4. Behind, but not above/below
        if isct.ylim[0] > self.ylim[0]:
            volumes.append( Cuboid( self.xlim, [self.ylim[0], isct.ylim[0]-1], isct.zlim, self.switch ) )

        # 5. Right, but not above/below or front/behind
        if isct.xlim[1] < self.xlim[1]:
            volumes.append( Cuboid( [isct.xlim[1]+1, self.xlim[1]], isct.ylim, isct.zlim, self.switch ) )

        # 6. Left, but not above/below or front/behind
        if isct.xlim[0] > self.xlim[0]:
            volumes.append( Cuboid( [self.xlim[0], isct.xlim[0]-1], isct.ylim, isct.zlim, self.switch ) )

        return volumes

In [7]:
def format_data( text ):
    lines = text.split("\n")
    steps = []
    for line in lines:
        parts = line.split(" ")
        switch = parts[0]

        limits = parts[1].split(",")
        xlim = [int(val) for val in limits[0][2:].split("..")]
        ylim = [int(val) for val in limits[1][2:].split("..")]
        zlim = [int(val) for val in limits[2][2:].split("..")]

        steps.append( Cuboid( xlim, ylim, zlim, switch ) )

    return steps

In [8]:
# reactor status will be a list of Cuboids that are switched on
reactor_status = []

In [9]:
# directions will be a list of Cuboids that have switch on or off
steps = format_data(mini)
steps

[on x=10..12,y=10..12,z10..12,
 on x=11..13,y=11..13,z11..13,
 off x=9..11,y=9..11,z9..11,
 on x=10..10,y=10..10,z10..10]

In [10]:
def apply_step( step, reactor_status, verbose = False ):

    new_reactor = []
    for idx, cube in enumerate( reactor_status ):
        isct = cube.intersect( step )
        if verbose:
            print("cube:", cube)
            print("isct:", isct)

        if isct is None:
            new_cubes = [cube]
        else:
            new_cubes = cube.remove_intersection( isct )

        if verbose:
            print( "Adding", len(new_cubes), "new cuboids:", idx )
            print(new_cubes)
            print("\n")
        # remove interesction since that will now be included in the new step's cuboid
        new_reactor += new_cubes

    # add on switch to reactor status
    if step.switch == "on":
        new_reactor.append(step)

    # for off switch, nothing to do
    # reactor_status only needs to include on positions, and we removed those in the for loop

    return new_reactor

In [11]:
def check_all_intersections( cuboid_list, verbose = False ):
    okay = True
    for i in range(len(cuboid_list)):
        for j in range(i+1, len(cuboid_list)):
            isct = cuboid_list[i].intersect( cuboid_list[j] )
            if isct is not None:
                okay = False
                if verbose:
                    print(i,j, isct)

    return okay

def count_on_positions( cuboid_list ):
    return sum( [x.size() for x in cuboid_list if x.switch == "on"] )

In [12]:
def apply_all_steps( steps ):
    reactor_status = []
    for i, step in enumerate(steps):
        reactor_status = apply_step(step, reactor_status)
        assert check_all_intersections, f'''Reactor Cuboids are not disjoint <on step {i}>'''

    return reactor_status

In [13]:
def get_initialization_directions( steps ):
    init_steps = []
    for step in steps:
        limit = step.xlim + step.ylim + step.zlim
        if min(limit) >= -50 and max(limit) <= 50:
            init_steps.append(step)
    return init_steps

In [14]:
reactor_status = []

In [15]:
reactor_status = apply_step( steps[0], reactor_status)
print( count_on_positions( reactor_status ) )
check_all_intersections( reactor_status )

27


True

In [16]:
reactor_status = apply_step( steps[1], reactor_status)
print( count_on_positions( reactor_status ) )
check_all_intersections( reactor_status )

46


True

In [17]:
reactor_status = apply_step( steps[2], reactor_status )
print( count_on_positions( reactor_status ) )
check_all_intersections( reactor_status )

38


True

In [18]:
reactor_status = apply_step( steps[3], reactor_status)
print( count_on_positions( reactor_status ) )
check_all_intersections( reactor_status )

39


True

In [19]:
test_steps = format_data(test)
init_test_steps = get_initialization_directions(test_steps)
print(len(test_steps), len(init_test_steps))
reactor_status = apply_all_steps( init_test_steps )
count_on_positions( reactor_status )

22 20


590784

In [20]:
data_steps = format_data(data)
init_data_steps = get_initialization_directions(data_steps)
print(len(data_steps), len(init_data_steps))
reactor_status = apply_all_steps( init_data_steps )
count_on_positions( reactor_status )

420 20


581108

# Part 2

In [21]:
with open("../data/2021/22_test.txt", "r") as f:
    test2 = f.read()

reactor_status = apply_all_steps( format_data(test2) )
count_on_positions( reactor_status )

2758514936282235

In [22]:
reactor_status = apply_all_steps( format_data(data) )
count_on_positions( reactor_status )

1325473814582641